## Mistral-7B-OpenOrca-GGUF

### Uso de modelo

In [7]:
import pandas as pd

# Opción que deshabilita el limite de columnas y filas mostradas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
from termcolor import cprint

In [9]:
path_data = '../data/clean'
df = pd.read_parquet(f'{path_data}/reviews_darden.parquet')
df = df[df['stars'] >= 4]  # filtrar por reviews positivos
df = df[['name', 'text']]

In [10]:
# Filtrar una muestra
df = df.sample(4)

---
#### version Q5

In [11]:
from ctransformers import AutoModelForCausalLM

model_Q5 = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", 
                                        model_file="mistral-7b-openorca.Q5_K_M.gguf",
                                        model_type="mistral",
                                        max_new_tokens=256,
                                        reset=True,
                                        gpu_layers=0
                                        )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
def get_prompt(review):
    prompt = f"""
    You are a humble restaurant manager that uses casual language. 
    You are talking to a customer. Do not call customer by their name. 
    With a brief one sentence, reply to this Yelp review posted by the customer: 
    {review}
    """
    return prompt

In [13]:
nombres_cadenas = df['name'].tolist()
reviews = df['text'].tolist()

cadena_reviews = zip(nombres_cadenas, reviews)

for name, review in cadena_reviews:
    print('\n\n')
    cprint(name, 'blue', attrs=['bold'])
    print(f'>> Review:\n{review}\n\n>> Reply: ')
    
    for text in model_Q5(get_prompt(review), stream=True):
        print(text, end='', flush=True)




Seasons 52
>> Review:
Busy loud and at 5:30. Flat bread pizza is terrific
 Lobster on flatbread! Soup eh. Salad with fried green tomatoes delicious. Needs a better craft beer menu.
>> Reply: 

    "Thanks for sharing your thoughts, buddy! We appreciate the feedback and will work on improving our soup and expanding our beer menu!"


LongHorn Steakhouse
>> Review:
My husband and I have traveled the world.  We are in Tampa now.  While visiting the area we decided to stop in and check out this restaurant.  What a wonderful dining evening meal.  We would rate this restaurant as one of the top dining experiences we have had anywhere.  Jackie took care of us giving us a "you are family" experience.  She gets a 11 on a 1 - 10 scale.  Our food was perfectly prepared and very reasonably priced.  We will make it a point to come back to the Longhorn Restaurant in Tampa, Florida.
>> Reply: 

    Oh, thanks for the kind words! We're glad you enjoyed your meal with us. We look forward to seeing yo

### Prueba de prompts (prevenir traducciones)

In [14]:
test_review = "Comida mala, no vuelvo."

test_prompt = f'''
You are a humble restaurant manager. 
You are talking to a Yelp user using their language. 
With a brief one sentence reply to this review posted by the user: 
"{test_review}"
'''

for text in model_Q5(test_prompt, stream=True):
    print(text, end="", flush=True)

Translation: "Bad food, won't come back."
Your reply would be:
"I apologize for your disappointment; we strive to improve and appreciate your feedback. We hope to see you again soon."